# Building LLM-based applications for clinician support

In this notebook, we will build an application to provide patient treatment recommendations for clinicians with the help of Large Language Models (LLMs).

To overcome LLMs limitations of hallucinations and lack of specific domain knowledge, we will allow the LLM to learn domain knowledge from context (In-Context Learning) by feeding into the prompts relevant texts in published papers about recommendations for disease treatment and maintenance.

There are two parts in this tutorial:
- Build a simple QARetrievalFromSource Chain, which takes in a user question, retrieve relevant documents from a vectorstore and ground the LLM to answer the question based on given context.

- Build an autonomous agent which perform multiple steps (search, calculation, query from database, etc) to provide a recommendation based on a patient's calculated eGFR score. The agent follows ReAct reasoning framework (Think-Act-Observe) and is given a diverse toolkit (MRKL) to choose from.

# First let's install the packages and dependencies required.

- langchain: Main package for building LLM application
- openai: For use OpenAI LLM model as engine for application
- faiss-cpu: Facebook AI Similarity Search. Vector Database for document retrieval.
- tiktoken: Package for counting tokens consumed by LLMs. Support several LLMs including OpenAI and HuggingFace models.
- pypdf & PyMuPDF: Package for loading PDF documents.

In [ ]:
%pip install --quiet langchain openai faiss-cpu tiktoken pypdf PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


We will create a working folder iaim_langchain and set this as the MAIN working directory

In [ ]:
%mkdir iaim_langchain
%cd /content/iaim_langchain

import os
MAIN_DIR = os.getcwd()

/content/iaim_langchain


# Let's download and extract the data folder which contains the following:
1. PDF files to be used for creating vectorstore.

2. Dummy Patient Database file (patient_data.csv).

3. Authentification json file: Containing OpenAI API keys and SerpAPI API Key

In [ ]:
!pip3 install --upgrade gdown
!gdown https://drive.google.com/uc?id=10TjeOFh7Doq8IvynqcYlCC2LwANUhDL8
!unzip iaim_data_files.zip

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Downloading...
From: https://drive.google.com/uc?id=10TjeOFh7Doq8IvynqcYlCC2LwANUhDL8
To: /content/iaim_langchain/iaim_data_files.zip
100% 16.7M/16.7M [00:00<00:00, 55.7MB/s]
Archive:  iaim_data_files.zip
  inflating: api_keys.json           
   creating: database/
  inflating: database/patients_info.csv  
   creating: raw_data/
  inflating: raw_data/ACG 2015 Hereditary GI Cancers.pdf  
  inflating: raw_data/AGA 2021 IBD CRC Surveillance.pdf  
  inflating: raw_data/ASCRS 2017 Lynch Syndrome.pdf  
  inflating: raw_data/Chronic kidney disease in the elderly.pdf  
  inflating: raw_data/ckd_evaluation_classification_stratification.pdf  
  inflating: raw_data/MSTF 2017 CRC Screening.pdf  
  inflating: raw_data/nihms-1069237.pdf  
  inflating: raw_data/nihms639366.pdf  
  inflating: raw_data/umj7603-154.pdf  
  inflating: raw_data/USPSTF 2021 CRC

# Import necessary packages

In [ ]:
import os, json, logging, re
from getpass import getpass
from pydantic import root_validator
from typing import Any, Union, Tuple, Dict, Callable, List, Optional, Literal
from pprint import pprint
import math, numexpr
from random import randint

from langchain import OpenAI
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import VectorStore, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.chains import RetrievalQAWithSourcesChain, LLMMathChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser, initialize_agent, load_tools, create_csv_agent
from langchain.tools import BaseTool
from langchain.chains.qa_with_sources.base import BaseQAWithSourcesChain
from langchain.base_language import BaseLanguageModel
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.tools.python.tool import PythonAstREPLTool # Need Python 3.9 and above
from langchain.agents.mrkl.base import ZeroShotAgent

- To use OpenAI models, we need a working API-Key. You can register for your own key at https://openai.com/blog/openai-api. <br> On your first registration, you will be given some free credits to try out the API. <br> Afterwards, OpenAI charges users based on tokens consumed. <br> Note that for free version, you can only access gpt-3.5-turbo model. For access to the largest model (gpt-4), you need to pay at least once.

- In addition, we will also need SerpAPI key to use it's API for Internet Search. You can register a free API Key here (https://serper.dev/).

- In this tutorial, we have provided you with the OPENAI API and SERP API keys so you don't need to worry about it. <br>

Note: It is recommended that you register API Keys inside your environment variables instead of hard-coding it inside your codes. This is to prevent others accidentally have access to your paid API keys

In [ ]:
with open("api_keys.json", "r") as f:
    keys = json.load(f)
    os.environ["OPENAI_API_KEY"] = keys["OPENAI_API_KEY"]
    os.environ["SERPER_API_KEY"] = keys["SERP_API"]

# Alternatively, you can manually assign your API keys to environment variables
# os.environ["OPENAI_API_KEY"] = "Your OpenAI_API_Key"
# os.environ["SERPER_API_KEY"] = "Your SERPER_API_KEY"

In [ ]:
def check_documents_token(docs: List[Document], llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")):
    if not isinstance(docs, List):
        docs = [docs]
    combine_document_chain = StuffDocumentsChain(
        llm_chain=LLMChain(llm=llm, prompt=PromptTemplate(template="{summaries}", input_variables=["summaries"]), verbose=False),
        verbose=False)
    return combine_document_chain.prompt_length(docs)

# Langchain

- Langchain is a Python/JS package for building LLM applications. It provides a modular framework to make interacting with LLM easier, including prompt engineering, processing inputs/outputs for chaining multiple LLM calls together.

- Langchain has integrations with many LLM models including OpenAI, Google and Hugging Face models. It also has extensive set of tools, API connectors as well as a variety of vector database (Pinecone, Chroma, FAISS, Redis, Elastic Search, etc)

- More information can be found at https://python.langchain.com/docs/get_started/introduction.html

# Creating the first Prompt

For prompt construction, Langchain used PromptTemplate module to allow reproducability with several LLM calls.

The text inside curly brackets {} in a string prompt acts as placeholder variables and will allow user to enter a different query each time to the prompt template while keeping the other parts same.

Let's setup a simple custom PromptTemplate to answer general questions. Here we add the sentence "Do not make up an answer if you dont know" to reduce the chance that LLM will hallucinate and make up answers.

In [ ]:
prompt_template = """Answer the question below. Do not make up an answer if you dont know.
Question: {question}
"""
basic_prompt = PromptTemplate(template=prompt_template, input_variables=["question"])
basic_chain = LLMChain(llm=ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo"), prompt=basic_prompt, verbose = True)

In [ ]:
basic_chain("Who is the prime minister of Singapore in 2021?")



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question below. Do not make up an answer if you dont know.
Question: Who is the prime minister of Singapore in 2021?


> Finished chain.


{'question': 'Who is the prime minister of Singapore in 2021?',
 'text': 'As of my last update in September 2021, the Prime Minister of Singapore is Lee Hsien Loong. However, please note that political positions can change, and it is always a good idea to verify the most up-to-date information.'}

As you can see, the text inside curly brackets {question} has been replaced with our query "Who is the prime minister of Singapore in 2021?". The LLM managed to answer the general question correctly.

In [ ]:
basic_chain("Who is the current minister of Singapore in 2023?")



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question below. Do not make up an answer if you dont know.
Question: Who is the current minister of Singapore in 2023?


> Finished chain.


{'question': 'Who is the current minister of Singapore in 2023?',
 'text': "I'm sorry, but as an AI language model, I don't have access to real-time information or the ability to browse the internet. Therefore, I cannot provide you with the current minister of Singapore in 2023. It's best to refer to official government sources or news outlets for the most up-to-date information."}

GPT models are trained on data up until 2021. Hence while it can answer question related to facts in year 2021, it cannot answer question after that. This is a limitation of LLM, as it's knowledge is bounded by the data it's trained on. <br>

Let's try again. Here we will ask the LLM to answer question about an older Deep Learning model (BERT - 2017) and a recent model (LLAMA-2 - 2023).

In [ ]:
basic_chain("Can you give a brief summary of BERT?")



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question below. Do not make up an answer if you dont know.
Question: Can you give a brief summary of BERT?


> Finished chain.


{'question': 'Can you give a brief summary of BERT?',
 'text': 'BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained natural language processing model developed by Google. It is designed to understand the context and meaning of words in a sentence by considering the words that come before and after them. BERT uses a transformer architecture, which allows it to capture long-range dependencies in text. It has been trained on a large amount of unlabeled text data from the internet, enabling it to learn general language representations. BERT has achieved state-of-the-art performance on various natural language processing tasks, such as question answering, sentiment analysis, and text classification.'}

In [ ]:
basic_chain("Can you give a brief summary of LLAMA-2?")



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question below. Do not make up an answer if you dont know.
Question: Can you give a brief summary of LLAMA-2?


> Finished chain.


{'question': 'Can you give a brief summary of LLAMA-2?',
 'text': "I'm sorry, but as an AI language model, my responses are generated based on a mixture of licensed data, data created by human trainers, and publicly available data. I have not been directly trained on specific proprietary databases or have access to classified information. Therefore, I don't have information on LLAMA-2."}

# Question Answering (QA) over local documents

To overcome the knowledge bound limitations of LLMs, we can:
1. Fine-Tune the LLM with latest data
2. In-Context Learning: Provide the LLM with the updated information inside the prompt.

Fine-Tuning is effective but is also very costly in terms of computation and data preparation. In addition, it doesn't solve the problem of un-updated knowledge base, as models fine-tuned today may be outdated a year later and need to be fine-tuned again.

In-Context Learning hence is an attractive solution to allows LLMs to have access to latest data at the point of running.

Next we will build a simple Chain which allows LLM to interact will a document.

Before that, let's ask the previous LLM chain a specific medical question.

In [ ]:
basic_chain("What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?")



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question below. Do not make up an answer if you dont know.
Question: What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?


> Finished chain.


{'question': 'What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?',
 'text': "I'm sorry, but as an AI language model, I don't have access to personal medical records or the ability to provide specific medical advice. It is important to consult with a healthcare professional or a gastroenterologist who can evaluate your specific situation and provide appropriate recommendations for colonoscopy screening based on your medical history and family history."}

LLM inherent knowledge tends to be more generic and hence it is not able of answering very specific question which requires strong domain knowledge. <br> We will provide it with some help from a research paper: **"Colorectal Cancer Screening: Recommendations for Physicians and Patients From the U.S Multi-Society Task Force on Colorectal Cancer"** - Douglas et al, 2017


In [ ]:
sample_document = os.path.join(".", "raw_data", "MSTF 2017 CRC Screening.pdf")
pages = PyMuPDFLoader(sample_document).load()
print("Number of pages:", len(pages))
sample_page = pages[8]
print(sample_page.page_content[-1000:])

Number of pages: 17
eﬁned
inherited syndrome caused by mutations in 1 or more
mismatch repair genes. Patients in families that meet the
clinical criteria for hereditary nonpolyposis CRC but have
microsatellite-stable CRCs have family colon cancer syn-
drome X, which has not been genetically deﬁned.122 Persons
in families with syndrome X should undergo colonoscopy at
least every 3 to 5 years, beginning 10 years before the age at
diagnosis of the youngest affected relative.
A family history of CRC in a ﬁrst-degree relative in-
creases the risk of CRC regardless of the age at diagnosis of
Table 4.Multi-Society Task Force Ranking of Current
Colorectal Cancer Screening Tests
Tier 1
Colonoscopy every 10 years
Annual fecal immunochemical test
Tier 2
CT colonography every 5 years
FIT–fecal DNA every 3 years
Flexible sigmoidoscopy every 10 years (or every 5 years)
Tier 3
Capsule colonoscopy every 5 years
Available tests not currently recommended
Septin 9
- 2017
MSTF Recommendations for CRC Scre

Let's initiate a QA chain to load the document in and answer our question.

Here we will use chain_type = "stuff", meaning that we simply "stuff" all the information inside all documents inside our placeholder **{context}**.

Let's investigate the prompt template.

In [ ]:
qa_stuff_chain = load_qa_chain(llm=OpenAI(temperature=0, max_tokens=512), chain_type="stuff", verbose=True)
print(qa_stuff_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [ ]:
answer = qa_stuff_chain({"input_documents": [sample_page],
          "question": "What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?"})

print("Answer:", answer["output_text"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

opportunistic setting is ensuring repeated annual perfor-
mance. Using a tier 1 approach that focuses on 2 tests makes
the discussion of CRC screening tests between physician and
patient manageable and feasible, and, as noted above,
expanding the number of options in the initial discussion
beyond 2 did not increase screening rates.19 Colonoscopy
and FIT can be adapted readily to either the sequential offer
of screening (colonoscopy is offered ﬁrst with FIT reserved
for those who decline colonoscopy), the multiple-options
approach (colonoscopy and FIT are each discussed with
patients, and if both are declined the discussion moves
sequentially to tier 2 tests), and the risk-stratiﬁed approach
(eg, colonoscopy is offered ﬁr

Great. It seems that the LLM has find the information we want from the piece of context given. <br>

In [ ]:
print("Number of characters for Page 10: ", len(sample_page.page_content))
print("Number of tokens for Page 10: ", check_documents_token(sample_page))

Number of characters for Page 10:  5549
Number of tokens for Page 10:  1286


- The tokens used by OpenAI LMs and most current LLMs are sub-word levels. This helps solve Out-Of-Vocabulary problems and reduce the number of model parameters required.
- As LLMs have maximum context length which limits the number of tokens which can be fed inside a single call, we usually should keep track of the number of tokens inside our prompts. Also, tokens cost $$$ so it's good to monitor the usage.

In [ ]:
answer = qa_stuff_chain({"input_documents": pages,
          "question": "What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?"})

print("Answer:", answer["output_text"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Colorectal Cancer Screening: Recommendations for Physicians
and Patients From the U.S. Multi-Society Task Force on
Colorectal Cancer
Douglas K. Rex,1 C. Richard Boland,2 Jason A. Dominitz,3 Francis M. Giardiello,4
David A. Johnson,5 Tonya Kaltenbach,6 Theodore R. Levin,7 David Lieberman,8 and
Douglas J. Robertson9
1Indiana University School of Medicine, Indianapolis, Indiana; 2University of California San Diego, San Diego, California;
3VA Puget Sound Health Care System, University of Washington, Seattle, Washington; 4Johns Hopkins University School of
Medicine, Baltimore, Maryland; 5Eastern Virginia Medical School, Norfolk, Virginia; 6San Francisco Veterans Affairs Medical
Center, San Francisco, California; 7Kaiser Perma

InvalidRequestError: ignored

Ooops!!! When we feed several pages inside our QA chains, the LLM breaks. <br> This is because the prompt length + maximum_tokens_generated exceed the LLM maximum_context_length. <br>

Let's explore a few ways which we can engineered around this LLM limitation.

# Divide And Conquer.

- Since we cannot process everything at once, we can break the task into smaller tasks, which we ask the LLM to answer on each document individually. All the individual answers are then combined to generate the final answer.
- Let's try to create a qa_chain with chain_type = **map_reduce**. Let's also ask the chain to return the intermediate answers.

In [ ]:
qa_mapreduce_chain = load_qa_chain(
    llm=OpenAI(temperature=0, max_tokens=512),
    chain_type="map_reduce",
    return_intermediate_steps=True)

answer = qa_mapreduce_chain({"input_documents": pages,
          "question": "What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?"})

print("Answer:\n" + answer["output_text"] + "\n")

for idx, intermediate_step in enumerate(answer["intermediate_steps"]):
    print(f"Document No {idx + 1} Answer:", intermediate_step)

Answer:
 Persons with a history of CRC or a documented advanced adenoma in a first-degree relative age <60 years or 2 first-degree relatives with these findings at any age are recommended to undergo screening by colonoscopy every 5 years, beginning 10 years before the age at diagnosis of the youngest affected relative, or at age 40, whichever is earlier.

Document No 1 Answer:  Persons with a family history of CRC or a documented advanced adenoma in a ﬁrst-degree relative age <60 years or 2 ﬁrst-degree relatives with these ﬁndings at any age are recommended to undergo screening by colonoscopy every 5 years, beginning 10 years before the age at diagnosis of the youngest affected relative or age 40, whichever is earlier.
Document No 2 Answer:  No relevant text.
Document No 3 Answer:  We recommend that clinicians offer CRC screening beginning at age 50 (strong recommendation, high-quality evidence). (See below for adjustments in recommended age for onset of screening based on race and fam

Great. The LLM manages to run without exceeding tokens limit. The answer is also accurate. <br> We can also return the intermediate answers to debug LLM's behaviour.

# Semantic Search with Vectorstore Database

Alternatively, we can implement a vector database to store all of our documents and only retrieve the relevant documents to our question. <br>
There are several advantages of this approach:
- Documents can be stuffed inside a single prompt. This will help preserve the documents granularity as compared to combining answers in a multi-stage process.
- Reduce tokens consumption. Divide-And-Conquer technically still runs through all documents, even for completely irrelevant documents.
- Higher quality context helps LLM focus on the more important text and reduce hallucinations.
- Can also be combined with Divide-And-Conquer strategy.

## Create vector index database

We will create a vector index database using Facebook AI Similarity Search Database. Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.<br>

We use OpenAIEmbeddings model to convert the texts into the vector representations which will be used for cosine similarity search

Typically, the loaded documents are further split into smaller text chunks with some overlapping window. The size of the text chunk is a parameter for tuning. Try not to use a very large chunk as you may run into maximum context length exceeding.

In [ ]:
document_files = [os.path.join(MAIN_DIR, "raw_data", path) for path in os.listdir(os.path.join(MAIN_DIR, "raw_data"))]
documents = []
for doc_file in document_files:
     documents.extend(PyMuPDFLoader(doc_file).load()) # Convert all pdf files into list of documents

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500) # Split document into smaller chunks. Each chunk containing ~600-700 tokens
texts = text_splitter.split_documents(documents)
print("Number of text chunks:", len(texts))

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)
docsearch.save_local(os.path.join(MAIN_DIR, "vectorstore"))
print("Document Store saved successfully!!!")

Number of text chunks: 1609


RateLimitError: ignored

Creation of new vector database requires running all documents through Embeddings model. This cost tokens & $$$. <br>

Once you have the database, simply load it instead of recreating the database again.

In [ ]:
docsearch = FAISS.load_local(os.path.join(MAIN_DIR, "vectorstore"), OpenAIEmbeddings())
print("Document Store loaded successfully!!!")

Document Store loaded successfully!!!


Now let's return the top 2 relevant documents w.r.t our previous question

In [ ]:
docsearch.similarity_search("What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?",
                            k=2)

[Document(page_content='Family history\nRecommended screening\nLynch Syndrome\nSee reference 34\nFamily Colon Cancer Syndrome X\nColonoscopy every 3-5 years beginning 10 years before the age at\ndiagnosis of the youngest affected relative\nColorectal cancer or an advanced adenoma in two\nﬁrst-degree relatives diagnosed at any age\nOR colorectal cancer or an advanced adenoma\nin a single ﬁrst-degree relative at age < 60 years\nColonoscopy every 5 years beginning 10 years before the age\nat diagnosis of the youngest affect interval or age 40,\nwhichever is earlier; for those with a single ﬁrst-degree\nrelative with colorectal cancer in whom no signiﬁcant\nneoplasia appears by age 60 years, physicians can offer\nexpanding the interval between colonoscopies\nColorectal cancer or an advanced adenoma in a\nsingle ﬁrst-degree relative diagnosed at\nage � 60 years\nBegin screening at age 40 years; tests and intervals are as\nper the average-risk screening recommendations (Table 4)\n10\nRex et 

## Question Answering with Vectorsearch

Now we will create a RetrievalQAWithSourcesChain which will integrate our QA from docs with the vector database. <br>

Under the hood, the retriever will retrieve/filter the most relevant documents from the database before stuffing it inside our context placeholder.

In [ ]:
qasource_chain_4 = RetrievalQAWithSourcesChain.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=512),
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k":4}), # k is the number of documents to retrieve from
    return_source_documents=True, # Return Source documents
    reduce_k_below_max_tokens=False, # If True, the chain will automatically select the number of documents to below max_tokens. Default to 3375 tokens.
    verbose=True
)

Let's Investigate the Prompt

In [ ]:
print(qasource_chain_4.combine_documents_chain.llm_chain.prompt.template)

Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the con

In [ ]:
answer_from_source = qasource_chain_4("What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?")



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


In [ ]:
pprint("Answer: "+answer_from_source["answer"])

('Answer: For patients with a history of family members having Colon Cancer '
 'Syndrome X, the recommended colonoscopy screening is every 3-5 years '
 'beginning 10 years before the age at diagnosis of the youngest affected '
 'relative. If there is a single first-degree relative with colorectal cancer '
 'at age <60 years, the recommended screening is every 5 years beginning 10 '
 'years before the age at diagnosis of the youngest affected relative or age '
 '40, whichever is earlier. If there is a single first-degree relative '
 'diagnosed at age ≥60 years, screening should begin at age 40 years and '
 'follow the average-risk screening recommendations. \n')


The answer is accurate. We can also check the document sources which the retriever feed to the LLMs.

In [ ]:
for idx, document in enumerate(answer_from_source["source_documents"]):
    print(f"Document {idx + 1}:", answer_from_source["source_documents"][0].metadata["source"].split("/")[-1])
    pprint(document.page_content)
    print("\n")

Document 1: MSTF 2017 CRC Screening.pdf
('Family history\n'
 'Recommended screening\n'
 'Lynch Syndrome\n'
 'See reference 34\n'
 'Family Colon Cancer Syndrome X\n'
 'Colonoscopy every 3-5 years beginning 10 years before the age at\n'
 'diagnosis of the youngest affected relative\n'
 'Colorectal cancer or an advanced adenoma in two\n'
 'ﬁrst-degree relatives diagnosed at any age\n'
 'OR colorectal cancer or an advanced adenoma\n'
 'in a single ﬁrst-degree relative at age < 60 years\n'
 'Colonoscopy every 5 years beginning 10 years before the age\n'
 'at diagnosis of the youngest affect interval or age 40,\n'
 'whichever is earlier; for those with a single ﬁrst-degree\n'
 'relative with colorectal cancer in whom no signiﬁcant\n'
 'neoplasia appears by age 60 years, physicians can offer\n'
 'expanding the interval between colonoscopies\n'
 'Colorectal cancer or an advanced adenoma in a\n'
 'single ﬁrst-degree relative diagnosed at\n'
 'age � 60 years\n'
 'Begin screening at age 40 years;

In [ ]:
print("Total number of tokens from retrieved docs:", check_documents_token(answer_from_source["source_documents"]))

Total number of tokens from retrieved docs: 1458


Let's try another example

In [ ]:
answer_from_source = qasource_chain_4("For Lynch syndrome (LS) patients, what is the guidelines for screening for gastric cancer? What is the guidelines for Endometrial cancer?")



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


In [ ]:
pprint("Answer: "+answer_from_source["answer"])

('Answer: For Lynch syndrome (LS) patients, the guidelines for screening for '
 'gastric cancer recommend baseline esophagogastroduodenoscopy (EGD) with '
 'gastric biopsy at age 30-35 years, and treatment of H. pylori infection when '
 'found. Ongoing surveillance every 3-5 years may be considered if there is a '
 'family history of gastric or duodenal cancer. For endometrial cancer, '
 'screening should be offered to women at risk for or affected with LS by '
 'endometrial biopsy and transvaginal ultrasound annually, starting at age 30 '
 'to 35 years before undergoing surgery or if surgery is deferred.\n')


In [ ]:
for idx, document in enumerate(answer_from_source["source_documents"]):
    print(f"Document {idx + 1}:", answer_from_source["source_documents"][0].metadata["source"].split("/")[-1])
    pprint(document.page_content)
    print("\n")

Document 1: ACG 2015 Hereditary GI Cancers.pdf
('Hereditary GI Cancer Syndromes\n'
 '© 2015 by the American College of Gastroenterology \n'
 'The American Journal of GASTROENTEROLOGY\n'
 ' \n'
 '225\n'
 ' Table 2  .  Summary of recommendations \n'
 ' Lynch syndrome (LS) \n'
 '  1. In individuals at risk for or affected with LS, screening for '
 'colorectal cancer by colonoscopy should be performed at least every 2 years, '
 'beginning \n'
 'between ages 20 and 25 years. Annual colonoscopy should be considered in '
 'conﬁ rmed mutation carriers (strong recommendation, moderate quality of \n'
 'evidence for screening, and very low quality of evidence for annual '
 'surveillance and age of initiation). \n'
 '  2. Colectomy with ileorectal anastomosis (IRA) is the preferred treatment '
 'of patients affected with LS with colon cancer or colonic neoplasia not '
 'controllable \n'
 'by endoscopy. Segmental colectomy is an option in patients unsuitable for '
 'total colectomy if regular posto

# LLM as Planning Agent

In the second part of the tutorial, we will build an autonomous LLM-Agent capable of planning and execute complex tasks which requires multiple steps and multiple tools which extend agent's capability beyond LLM's.<br>

We will create an Agent which aims to provide recommendations for patients with Chronic Kidney Diseases (CKD). We want it to search for patients information, perform calculation, search internal knowledge base and external knowledge base (internet) to find information needed.

Let's first create the Custom Tools required for the agents to work with.
1. QASearchTool: Tool linked to our previously created RetrievalQAChain from last session.
2. MathTool: Tool to do math. LLM is known for it's poor capability of doing complex Math.
3. CSVSearchTool: Tool to perform query from table/SQL database to get patient's information.
4. SearchTool: Tool to search stuffs on the internet
5. GeneralKnowledgeTool: This is just an LLM Tool. MRKL agents tend to prefer using tools even in cases where it doesnt need to.

In [ ]:
class QASearchTool(BaseTool):
    name: str = "Docsearch QA Tool"
    description: str = "Use this tool to search for document and answer questions related to treatment of disease"
    llm: BaseLanguageModel
    k: int = 4
    docstore: str = os.path.join(MAIN_DIR, "vectorstore")

    @root_validator()
    def generate_qa_chain(cls, values):
        docsearch = FAISS.load_local(values["docstore"], OpenAIEmbeddings())
        print("Document Store loaded successfully!!!")
        values["chain"] = RetrievalQAWithSourcesChain.from_chain_type(
            llm=values["llm"],
            chain_type="stuff",
            retriever=docsearch.as_retriever(search_kwargs={"k":values["k"]}), # k is the number of documents to retrieve from
            return_source_documents=True,
            reduce_k_below_max_tokens=False, # If True, the chain will automatically select the number of documents to below max_tokens. Default to 3375 tokens.
            verbose=True
            )
        return values

    def _run(self, query):
        return self.chain(query)

    def _arun(self, query):
        return NotImplementedError

class CustomMathChain(LLMMathChain):
    fallback_tool = PythonAstREPLTool()
    def _evaluate_expression(self, expression: str) -> str:
        try:
            local_dict = {"pi": math.pi, "e": math.e}
            output = numexpr.evaluate(
                    expression.strip(),
                    global_dict={},  # restrict access to globals
                    local_dict=local_dict,  # add common mathematical functions
                )
        except Exception as e:
            try: # Fall back to python shell
                output = self.fallback_tool(expression.strip())
            except Exception as e:
                raise ValueError(
                    f'LLMMathChain._evaluate("{expression}") raised error: {e}.'
                    " Please try again with a valid numerical expression"
                )

        # Remove any leading and trailing brackets from the output
        return re.sub(r"^\[|\]$", "", str(output))

class CSVDatabaseSearchTool(BaseTool):
    name: str = "CSVDatabase Search"
    description: str = "Use this tool to search for information in a CSV database"
    llm: BaseLanguageModel
    data_path: str

    @root_validator()
    def generate_agent_chain(cls, values):
        values["agent"] = create_csv_agent(
            llm = values["llm"],
            path = values["data_path"],
            verbose = True,
            agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION
        )
        return values

    def _run(self, query):
        return self.agent(query)

    def _arun(self, query):
        return NotImplementedError

## Define Agent Toolkits

In [ ]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens=256)
math_chain = CustomMathChain.from_llm(llm=llm)
llm_chain = LLMChain(prompt=ChatPromptTemplate.from_messages([HumanMessagePromptTemplate.from_template("Question: {question}")]), llm=llm)
search = GoogleSerperAPIWrapper()

tools = [QASearchTool(k=4, llm=llm, docstore=os.path.join(MAIN_DIR, "vectorstore"), description = "Use this tool to search for document and answer questions related to treatment of disease"),
         CSVDatabaseSearchTool(
              name = "Patient Database Search",
              description = "Use this tool to search for patients information including demographics (Patient ID, age, gender, race) and lab results (Creatinine)",
              llm=llm, data_path=os.path.join(MAIN_DIR, "database", "patients_info.csv")),
         Tool(name='Calculator', func=math_chain.run, description='Useful for when you need to answer questions about math.'),
         Tool(name='General Knowledge',func=llm_chain.run, description='Useful for general knowledge question.'),
         Tool(name="Search", func=search.run, description="Useful for when you need to search for information online, especially math equations.")
         ]

Document Store loaded successfully!!!


## Define Agent Instructions

Let's create a prompt for agent behaviour. We will use **ReAct** framework (Think-Act-Observe) for agent planning/reasoning and **MRKL** framework for agent choice of tools/actions.

In Langchain, **MRKL** agent is implemented under ZeroShotAgent module

In [ ]:
prefix = """You are a helpful physician assistant giving recommendations on treatment for Chronic Kidney Disease (CKD).
However, you are terrible at remembering math equations and should always search it somewhere else.

For each patient, calculate the eGFR score for the patient provided based on the latest formula.
Remember to convert creatinine measure to the unit specified in the equation. Look for additional information of this patient from database search tools if needed.
Finally, based on this patient eGFR and demographics (age, gender, race), classify CKD stage and return the recommendations of the suitable treatment for chronic kidney disease.
You have access to the following tools:"""

suffix = """Begin!

Patient Query: {input}
Thought: {agent_scratchpad}"""

AGENT_PROMPT = ZeroShotAgent.create_prompt(tools=tools, prefix=prefix, suffix=suffix)
print(AGENT_PROMPT.template)

You are a helpful physician assistant giving recommendations on treatment for Chronic Kidney Disease (CKD).
However, you are terrible at remembering math equations and should always search it somewhere else.

For each patient, calculate the eGFR score for the patient provided based on the latest formula.
Remember to convert creatinine measure to the unit specified in the equation. Look for additional information of this patient from database search tools if needed.
Finally, based on this patient eGFR and demographics (age, gender, race), classify CKD stage and return the recommendations of the suitable treatment for chronic kidney disease.
You have access to the following tools:

Docsearch QA Tool: Use this tool to search for document and answer questions related to treatment of disease
Patient Database Search: Use this tool to search for patients information including demographics (Patient ID, age, gender, race) and lab results (Creatinine)
Calculator: Useful for when you need to answ

- Let's Initialize our Agent. For tasks required long-term planning and reasoning, it's recommended to use bigger LLM models, as it has been shown that in general, models with more parameters perform much better in Chain-Of-Thought reasoning and Multi-Step Planning.<br><br>
- To prevent Agent getting stuck forever & burn tokens, set max_iterations to terminate Agent loop.

In [ ]:
agent_llm_chain = LLMChain(llm=ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens=256), prompt=AGENT_PROMPT)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=agent_llm_chain, allowed_tools=tool_names)

agent_args = {"max_iterations": 10, "early_stopping_method": "generate"}
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, **agent_args)

Let's check if the Agent can choose the correct tools and perform the simple singular task

In [ ]:
agent_executor("How many episodes are there in Harry Potter series")



> Entering new AgentExecutor chain...
The patient's query is not related to CKD treatment but rather a general knowledge question about the Harry Potter series.
Action: General Knowledge
Action Input: How many episodes are there in Harry Potter series
Observation: The Harry Potter series is a set of seven books, not episodes. However, there are eight movies based on these books.
Thought:I now know the final answer
Final Answer: The Harry Potter series consists of seven books. However, there are eight movies based on these books.

> Finished chain.


{'input': 'How many episodes are there in Harry Potter series',
 'output': 'The Harry Potter series consists of seven books. However, there are eight movies based on these books.'}

In [ ]:
agent_executor("What is 300 square multiply by 15 plus 40?")



> Entering new AgentExecutor chain...
The patient is asking for a mathematical calculation. I can use the calculator tool to solve this.
Action: Calculator
Action Input: 300*300*15+40
Observation: Answer: 1350040
Thought:I now know the final answer.
Final Answer: The result of the calculation 300 square multiply by 15 plus 40 is 1350040.

> Finished chain.


{'input': 'What is 300 square multiply by 15 plus 40?',
 'output': 'The result of the calculation 300 square multiply by 15 plus 40 is 1350040.'}

In [ ]:
agent_executor("What is the top 5 countries on Men's FIFA ranking in 2023?")



> Entering new AgentExecutor chain...
This question is about the FIFA ranking in 2023, which is a general knowledge question. I need to use the search tool to find the latest information.
Action: Search
Action Input: Top 5 countries on Men's FIFA ranking in 2023
Observation: 1. Argentina ; 2. France ; 3. Brazil ; 4. England ; 5. Belgium ... FIFA World Rankings ; 3, Brazil (BRA), 1,828.27 ; 4, England (ENG), 1,797.39 ; 5, Belgium (BEL), 1,788.55 ; 6, Croatia (CRO), 1,742.55 ... FIFA Men's World Ranking List ; 53, Canada, 1431.64 ; 54, Slovakia, 1425.58 ; 55, Venezuela, 1406.1 ; 56, Finland, 1405.71 ... June 2023 FIFA World Rankings revealed: Spain, United States, Mexico, England, Argentina, Brazil... 5, Belgium (BEL), 1,788.55, 1,789, 5. 6, Croatia (CRO), 1,742.55, 1,743, 6. 7, Netherlands (NED), 1,731.23, 1,731, 7. 8, Italy (ITA), 1,726.58, 1,727, 8. The rankings were introduced in December 1992, and eight teams (Argentina, Belgium, Brazil, France, Germany, Italy, the Netherlands and

{'input': "What is the top 5 countries on Men's FIFA ranking in 2023?",
 'output': "The top 5 countries on Men's FIFA ranking in 2023 are 1. Argentina, 2. France, 3. Brazil, 4. England, 5. Belgium."}

In [ ]:
agent_executor("What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?")



> Entering new AgentExecutor chain...
First, I need to find out the recommended colonoscopy screening for Polyp patients with a family history of Colon Cancer Syndrome X.
Action: Docsearch QA Tool
Action Input: What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: {'question': 'What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?', 'answer': 'For Polyp patients with a history of family members having Colon Cancer Syndrome X, the recommended screening is a colonoscopy every 3-5 years. This should begin 10 years before the age at diagnosis of the youngest affected relative.\n', 'sources': '/content/iaim_langchain/raw_data/MSTF 2017 CRC Screening.pdf', 'source_documents': [Document(page_content='Family history\nRecommended screening\nLynch Syndrome\nSee re

{'input': 'What is the recommended colonoscopy screening for Polyp patients with history of family members having Colon Cancer Syndrome X?',
 'output': 'For Polyp patients with a history of family members having Colon Cancer Syndrome X, the recommended screening is a colonoscopy every 3-5 years. This should begin 10 years before the age at diagnosis of the youngest affected relative.'}

Awesome. It seems like Agent can choose the correct tool. <br>
Now let's allow it to work on the actual task which requires multiple steps.

In [ ]:
agent_executor("Recommend Treatment for patient Andy. Patient ID is 17238471")



> Entering new AgentExecutor chain...
First, I need to find out the patient's demographic information and lab results.
Action: Patient Database Search
Action Input: 17238471

> Entering new AgentExecutor chain...
Thought: The question is asking for the row of the dataframe where the Patient ID is 17238471.
Action: I will use the pandas dataframe method `df.loc[]` to locate the row where the Patient ID is 17238471.
Action Input: `df.loc[df['Patient ID'] == 17238471]`
Observation: I will use the pandas dataframe method `df.loc[]` to locate the row where the Patient ID is 17238471. is not a valid tool, try another one.
Thought:I need to use the python_repl_ast tool to execute the python command.
Action: python_repl_ast
Action Input: df.loc[df['Patient ID'] == 17238471]
Observation:    No  Name  Patient ID      NRIC       DOB Gender Ethnicity  \
0   1  Andy    17238471  G1234567  1/2/1950   Male     Black   

   Creatinine Serum    Unit  
0               111  umol/L  
Thought:I now know 

{'input': 'Recommend Treatment for patient Andy. Patient ID is 17238471',
 'output': 'The recommended treatment for Andy, who is at Stage 2 Chronic Kidney Disease, includes interventions to slow the progression of the disease such as strict glucose control in diabetes, strict blood pressure control, and angiotensin-converting enzyme inhibition or angiotensin-2 receptor blockade. It is also recommended to avoid potential nephrotoxins like nonsteroidal anti-inflammatory drugs and to monitor for complications of CKD such as hyperkalemia, metabolic acidosis, hyperphosphatemia, vitamin D deficiency, secondary hyperparathyroidism, and anemia.'}

In [ ]:
agent_executor("Recommend Treatment for patient Bobby. Patient ID is 12374872")



> Entering new AgentExecutor chain...
I need to find out more about Bobby's health status, especially his kidney function. 
Action: Patient Database Search
Action Input: Patient ID 12374872

> Entering new AgentExecutor chain...
Thought: I need to find the row in the dataframe where the Patient ID is 12374872.
Action: python_repl_ast
Action Input: df[df['Patient ID'] == 12374872]
Observation:    No   Name  Patient ID      NRIC       DOB Gender Ethnicity  \
1   2  Bobby    12374872  S2123871  8/3/1944   Male     Black   

   Creatinine Serum    Unit  
1               120  umol/L  
Thought:I now know the final answer
Final Answer: The patient with ID 12374872 is Bobby, a male of Black ethnicity born on 8/3/1944. His Creatinine Serum level is 120 umol/L.

> Finished chain.

Observation: {'input': 'Patient ID 12374872', 'output': 'The patient with ID 12374872 is Bobby, a male of Black ethnicity born on 8/3/1944. His Creatinine Serum level is 120 umol/L.'}
Thought:I need to calculate Bobb

{'input': "Human: Patient's Name is Bobby. Patient ID is 12374872",
 'output': 'Bobby is in Stage 3a of Chronic Kidney Disease. The recommended treatment for this stage includes management of hypertension, possibly with an angiotensin-converting enzyme inhibitor (ACE-I) or an angiotensin II receptor blocker (ARB), and management of diabetes, aiming for a goal hemoglobin A1c of ~ 7.0%. Dose adjustments in oral hypoglycemic agents may be necessary.'}

In [ ]:
agent_executor("Recommend Treatment for patient Emily. Patient ID is 19238498")



> Entering new AgentExecutor chain...
I need to find Emily's demographic information and lab results to calculate her eGFR score.
Action: Patient Database Search
Action Input: Patient ID 19238498

> Entering new AgentExecutor chain...
Thought: The question is asking for the row of the dataframe where the Patient ID is 19238498. I can use the pandas function `df.loc[]` to locate this row.
Action: python_repl_ast
Action Input: df.loc[df['Patient ID'] == 19238498]
Observation:    No   Name  Patient ID      NRIC        DOB  Gender Ethnicity  \
3   4  Emily    19238498  G8712837  13/2/1975  Female     White   

   Creatinine Serum    Unit  
3                80  umol/L  
Thought:I now know the final answer
Final Answer: The patient with ID 19238498 is Emily, a female of White ethnicity, born on 13/2/1975. Her Creatinine Serum level is 80 umol/L.

> Finished chain.

Observation: {'input': 'Patient ID 19238498', 'output': 'The patient with ID 19238498 is Emily, a female of White ethnicity, b

{'input': "Human: Patient's Name is Emily. Patient ID is 19238498.",
 'output': 'Emily is at Stage 2 Chronic Kidney Disease with an eGFR score of approximately 75.74. The recommended treatment includes interventions to slow the progression of kidney disease such as strict glucose control in diabetes, strict blood pressure control, and angiotensin-converting enzyme inhibition or angiotensin-2 receptor blockade. It is also recommended to avoid potential nephrotoxins like nonsteroidal anti-inflammatory drugs and to monitor for complications of CKD such as hyperkalemia, metabolic acidosis, hyperphosphatemia, vitamin D deficiency, secondary hyperparathyroidism, and anemia.'}

In [ ]:
agent_executor("Recommend Treatment for patient Johnny. Patient ID is 98665543")



> Entering new AgentExecutor chain...
I need to find the patient's information including demographics and lab results.
Action: Patient Database Search
Action Input: Patient ID 98665543

> Entering new AgentExecutor chain...
Thought: The question is not clear. It seems like the user wants to retrieve the row of the dataframe where the Patient ID is 98665543. However, without a clear question, I can't be sure. I will assume this is the case for now.
Action: python_repl_ast
Action Input: df[df['Patient ID'] == 98665543]
Observation: Empty DataFrame
Columns: [No, Name, Patient ID, NRIC, DOB, Gender, Ethnicity, Creatinine Serum, Unit]
Index: []
Thought:The dataframe does not contain a row where the Patient ID is 98665543.
Final Answer: There is no patient with the ID 98665543 in the dataframe.

> Finished chain.

Observation: {'input': 'Patient ID 98665543', 'output': 'There is no patient with the ID 98665543 in the dataframe.'}
Thought:The patient ID provided does not exist in the databa

{'input': "Human: Patient's Name is Daniel. Patient ID is 98665543.",
 'output': "I'm sorry, but there is no patient with the ID 98665543 in our database. Could you please check the ID again?"}

# 🏆 Congratulations

## Learning Resources

You can start your journey on building LLM applications with the following resources:

- [Langchain](https://python.langchain.com/docs/get_started/introduction.html): Main Documentation Page of Langchain<br>

- [Pinecone LLM Series](https://www.pinecone.io/learn/series/langchain/langchain-intro/): This is a series of tutorials created by the Pinecone team. They also have a series of video tutorials which are easy to follow and learn. Also a great source to learn about vector database.

- [OpenAI](https://platform.openai.com/docs/introduction): Documentations to use OpenAI API

- [HuggingFace](https://huggingface.co/docs/transformers/index): Using Open-Source LLM with Hugging Face Transformers